In [1]:
import os
import random
from datetime import date, timedelta
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date, Float
import sqlite3
import re
import ast
import json
import contextlib

In [2]:


def extract_sql_code_block(text: str) -> str:
    """
    Extracts SQL from markdown-style code blocks if present.
    If the input looks like a list of tuples in string form, it will:
    - Parse the list
    - Quote all inner values
    - Sort the list
    - Return a string of tuples joined by commas
    """
    text = text.strip()

    # Handle SQL code block: ```sql ... ```
    match = re.search(r"```sql\s*(.*?)\s*```", text, re.DOTALL | re.IGNORECASE)
    if match:
        sql = match.group(1).replace('\n', ' ').strip()
        return sql

    # Handle list-of-tuples-as-string case
    try:
        # Safely evaluate the string to a Python list
        parsed = ast.literal_eval(text)
        if isinstance(parsed, list) and all(isinstance(item, tuple) for item in parsed):
            # Convert all inner items to strings and quote them
            normalized = [
                tuple(str(x).strip() for x in row)
                for row in parsed
            ]
            # Sort for consistent comparison
            sorted_rows = sorted(normalized)
            return ", ".join(str(row) for row in sorted_rows)
    except Exception:
        pass

    # Fallback: remove brackets if it looks like a wrapped string
    if text.startswith("[") and text.endswith("]"):
        text = text[1:-1].strip().strip("'\"")

    return text.replace('\n', ' ').strip()

In [3]:
import getpass
import os

if "TOGETHER_API_KEY" not in os.environ:
    os.environ["TOGETHER_API_KEY"] = getpass.getpass("af437c26f93bd7d80c5d9af2c63e4e8c44886676503bc843cc133803892a41f0")

In [6]:
import os
os.environ["TOGETHER_API_KEY"] = "af437c26f93bd7d80c5d9af2c63e4e8c44886676503bc843cc133803892a41f0"

In [7]:
os.environ["TOGETHER_API_KEY"]

'af437c26f93bd7d80c5d9af2c63e4e8c44886676503bc843cc133803892a41f0'

In [8]:
from langchain_together import ChatTogether

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [9]:
llm.invoke("who are you?").content

'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="togethercomputer/llama-3-70b",
    openai_api_base="https://api.together.xyz/v1",
    openai_api_key="your_together_ai_api_key"
)

In [3]:
# Tugberk Groq API Key
#GROQ_API_KEY=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
#-------------
#GROQ_API_KEY=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
# GROQ_API_KEY=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
#--------Shivam
#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

In [ ]:
os.environ["GROQ_API_KEY"]="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"

In [ ]:
from langchain_groq import ChatGroq

#os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
#os.environ['GROQ_API_KEY'] = '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'

#llm = ChatGroq(model='llama-3.3-70b-versatile')

#print(llm.invoke('who are you?').content)

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

In [10]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:////Users/onurcanmemis/Downloads/spider_data/database/concert_singer/concert_singer.sqlite")
print(db.dialect)
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

sqlite


[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11747c940>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11747c940>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11747c940>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [11]:
for i in tools:
    print(i.name)
    print(i.description)

sql_db_query
Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema
Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
sql_db_list_tables
Input is an empty string, output is a comma-separated list of tables in the database.
sql_db_query_checker
Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [12]:
table_names = db.get_usable_table_names()
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer.
You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

The tables in the database is given as {table_names}. To start you should ALWAYS look at the tables and their columns in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect="SQLite",
    table_names=", ".join(table_names)
)
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, prompt=system_message)

In [33]:
import ast

def extract_sql_and_result_from_agent_output(agent_output: dict) -> dict:
    """
    Extract the final SQL query and its returned value from LangGraph-style agent output.

    Returns a dictionary with:
        - 'gold_sql': SQL query as string
        - 'gold_result': formatted string of tuples
    """
    messages = agent_output.get("messages", [])

    sql_query = ""
    sql_result = ""

    for msg in messages:
        if hasattr(msg, "name") and msg.name == "sql_db_query":
            # This is the ToolMessage containing the SQL result
            raw_result = msg.content
            try:
                parsed = ast.literal_eval(raw_result)
                if isinstance(parsed, list) and all(isinstance(t, tuple) for t in parsed):
                    sorted_rows = sorted(tuple(str(i).strip() for i in row) for row in parsed)
                    sql_result = ", ".join(str(row) for row in sorted_rows)
            except Exception as e:
                sql_result = raw_result  # fallback to raw

        elif hasattr(msg, "tool_calls"):
            for tool_call in msg.tool_calls:
                if tool_call["name"] == "sql_db_query":
                    sql_query = tool_call["args"]["query"]

    return {
        "generated_sql": sql_query.strip(),
        "generated_result": sql_result.strip()
    }

In [25]:
answer=agent_executor.invoke(
            {"messages": [{"role": "user", "content": golden_data_easy[1]["question"]}]},
            config={"recursion_limit": 10}
        )

In [34]:
extract_sql_and_result_from_agent_output(answer)

{'generated_sql': 'SELECT COUNT(*) FROM singer', 'generated_result': "('6',)"}

In [10]:
events = agent_executor.stream(
    {"messages": [("user", golden_data_easy[0]["question"])]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

How many singers do we have?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_knnk)
 Call ID: call_knnk
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

concert, singer, singer_in_concert, stadium
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_8605)
 Call ID: call_8605
  Args:
    table_names: singer, singer_in_concert, concert, stadium
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE concert (
	"concert_ID" INTEGER, 
	"concert_Name" TEXT, 
	"Theme" TEXT, 
	"Stadium_ID" TEXT, 
	"Year" TEXT, 
	PRIMARY KEY ("concert_ID"), 
	FOREIGN KEY("Stadium_ID") REFERENCES stadium ("Stadium_ID")
)

/*
3 rows from concert table:
concert_I

## Hard Questions

In [36]:
import json
with open("extracted_sql_examples_hard.json", "r") as f:
    data = json.load(f)

golden_data_hard = [
    {"db_id": item["db_id"], "question": item["question"], "query": item["gold_sql"], "result": item["gold_result"]}
    for item in data
]

In [37]:
golden_data_hard[0]

{'db_id': 'concert_singer',
 'question': 'Show countries where a singer above age 40 and a singer below 30 are from.',
 'query': 'select country from singer where age > 40 intersect select country from singer where age < 30',
 'result': "('France',)"}

In [38]:
import json
import os
from tqdm import tqdm
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langgraph.prebuilt import create_react_agent

save_path = "hard_answers.json"

# Load previous progress if available
if os.path.exists(save_path):
    with open(save_path, "r") as f:
        hard_answers = json.load(f)
else:
    hard_answers = []

start_index = len(hard_answers)

# Wrap tqdm around the remaining examples
for i in tqdm(range(start_index, len(golden_data_hard)), desc="Generating SQL", unit="q"):
    item = golden_data_hard[i]
    try:
        print(item["question"])

        # Setup DB first (important: get tables *after* db is loaded)
        db = SQLDatabase.from_uri(f"sqlite:////Users/onurcanmemis/Downloads/spider_data/database/{item['db_id']}/{item['db_id']}.sqlite")
        table_names = db.get_usable_table_names()

        system_message = """
        You are an agent designed to interact with a SQL database.
        Given an input question, create a syntactically correct {dialect} query to run,
        then look at the results of the query and return the answer.
        You can order the results by a relevant column to return the most interesting
        examples in the database. Never query for all the columns from a specific table,
        only ask for the relevant columns given the question.

        You MUST double check your query before executing it. If you get an error while
        executing a query, rewrite the query and try again.

        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
        database.

        The tables in the database is given as {table_names}. To start you should ALWAYS look at the 
        tables and their columns in the database to see what you can query. Do NOT skip this step.

        Then you should query the schema of the most relevant tables.
        """.format(
            dialect="SQLite",
            table_names=", ".join(table_names)
        )

        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        tools = toolkit.get_tools()
        agent_executor = create_react_agent(llm, tools, prompt=system_message)

        answer = agent_executor.invoke(
            {"messages": [{"role": "user", "content": item["question"]}]},
            config={"recursion_limit": 10}
        )

        generated_query = extract_sql_and_result_from_agent_output(answer)["generated_sql"]
        generated_result = extract_sql_and_result_from_agent_output(answer)["generated_result"]

        hard_answers.append({
            "db_id": item["db_id"],
            "question": item["question"],
            "generated_sql": generated_query or "",
            "result": generated_result or "",
            "gold_sql": item["query"],
            "gold_result": item["result"]
        })

        print(f"✅ Success at index {i}")
        print(hard_answers[i])

        with open(save_path, "w") as f:
            json.dump(hard_answers, f, indent=2)

    except Exception as e:
        error_msg = str(e)
        if "GRAPH_RECURSION_LIMIT" in error_msg or "Recursion limit" in error_msg:
            print(f"⚠️ Recursion limit hit at index {i}. Skipping.")
            hard_answers.append({
                "db_id": item["db_id"],
                "question": item["question"],
                "generated_sql": "",
                "result": "",
                "gold_sql": item["query"]
            })
            print(hard_answers)
        elif "tool_use_failed" in error_msg or "table" in error_msg.lower():
            print(f"⚠️ Tool failure or table issue at index {i}. Skipping.")
            break
        else:
            print(f"❌ Error at index {i}: {e}")
            break

        with open(save_path, "w") as f:
            json.dump(hard_answers, f, indent=2)

Generating SQL:   0%|          | 0/15 [00:00<?, ?q/s]

Show countries where a singer above age 40 and a singer below 30 are from.


Generating SQL:   7%|▋         | 1/15 [00:17<04:08, 17.72s/q]

✅ Success at index 0
{'db_id': 'concert_singer', 'question': 'Show countries where a singer above age 40 and a singer below 30 are from.', 'generated_sql': 'SELECT T1.country FROM singer AS T1 INNER JOIN singer_in_concert AS T2 ON T1.id = T2.singer_id INNER JOIN concert AS T3 ON T2.concert_id = T3.id WHERE T1.age > 40 INTERSECT SELECT T1.country FROM singer AS T1 INNER JOIN singer_in_concert AS T2 ON T1.id = T2.singer_id INNER JOIN concert AS T3 ON T2.concert_id = T3.id WHERE T1.age < 30', 'result': 'Error: (sqlite3.OperationalError) no such column: T1.id\n[SQL: SELECT T1.country FROM singer AS T1 INNER JOIN singer_in_concert AS T2 ON T1.id = T2.singer_id INNER JOIN concert AS T3 ON T2.concert_id = T3.id WHERE T1.age > 40 INTERSECT SELECT T1.country FROM singer AS T1 INNER JOIN singer_in_concert AS T2 ON T1.id = T2.singer_id INNER JOIN concert AS T3 ON T2.concert_id = T3.id WHERE T1.age < 30]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)', 'gold_sql': 'select country fr

Generating SQL:  13%|█▎        | 2/15 [00:40<04:26, 20.47s/q]

...

✅ Success at index 12
{'db_id': 'course_teach', 'question': 'Show the hometowns shared by at least two teachers.', 'generated_sql': 'SELECT hometown FROM teacher GROUP BY hometown HAVING COUNT(hometown) > 1', 'result': '', 'gold_sql': 'select hometown from teacher group by hometown having count(*) >= 2', 'gold_result': ''}
Which cities do more than one employee under age 30 come from?


Generating SQL:  93%|█████████▎| 14/15 [04:02<00:17, 17.44s/q]

✅ Success at index 13
{'db_id': 'employee_hire_evaluation', 'question': 'Which cities do more than one employee under age 30 come from?', 'generated_sql': 'SELECT city FROM employee WHERE age < 30 GROUP BY city HAVING COUNT(*) > 1', 'result': "('Bath',)", 'gold_sql': 'select city from employee where age < 30 group by city having count(*) > 1', 'gold_result': "('Bath',)"}
Find the cities that have more than one employee under age 30.


Generating SQL: 100%|██████████| 15/15 [04:22<00:00, 17.48s/q]

✅ Success at index 14
{'db_id': 'employee_hire_evaluation', 'question': 'Find the cities that have more than one employee under age 30.', 'generated_sql': 'SELECT T1.city FROM shop AS T1 INNER JOIN employee AS T2 ON T1.id = T2.shop_id WHERE T2.age < 30 GROUP BY T1.city HAVING COUNT(T2.id) > 1', 'result': 'Error: (sqlite3.OperationalError) no such column: T1.city\n[SQL: SELECT T1.city FROM shop AS T1 INNER JOIN employee AS T2 ON T1.id = T2.shop_id WHERE T2.age < 30 GROUP BY T1.city HAVING COUNT(T2.id) > 1]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)', 'gold_sql': 'select city from employee where age < 30 group by city having count(*) > 1', 'gold_result': "('Bath',)"}


## Medium Questions

In [44]:
import json
with open("extracted_sql_examples_medium.json", "r") as f:
    data = json.load(f)

golden_data_medium = [
    {"db_id": item["db_id"], "question": item["question"], "query": item["gold_sql"], "result": item["gold_result"]}
    for item in data
]

In [45]:
golden_data_medium[0]

{'db_id': 'concert_singer',
 'question': 'What are all distinct countries where singers above age 20 are from?',
 'query': 'select distinct country from singer where age > 20',
 'result': "('Netherlands',), ('France',), ('United States',)"}

In [52]:
import json
import os
from tqdm import tqdm
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langgraph.prebuilt import create_react_agent

save_path = "medium_answers.json"

# Load previous progress if available
if os.path.exists(save_path):
    with open(save_path, "r") as f:
        medium_answers = json.load(f)
else:
    medium_answers = []

start_index = len(medium_answers)

# Wrap tqdm around the remaining examples
for i in tqdm(range(start_index, len(golden_data_medium)), desc="Generating SQL", unit="q"):
    item = golden_data_medium[i]
    try:
        print(item["question"])

        # Setup DB first (important: get tables *after* db is loaded)
        db = SQLDatabase.from_uri(f"sqlite:////Users/onurcanmemis/Downloads/spider_data/database/{item['db_id']}/{item['db_id']}.sqlite")
        table_names = db.get_usable_table_names()

        system_message = """
        You are an agent designed to interact with a SQL database.
        Given an input question, create a syntactically correct {dialect} query to run,
        then look at the results of the query and return the answer.
        You can order the results by a relevant column to return the most interesting
        examples in the database. Never query for all the columns from a specific table,
        only ask for the relevant columns given the question.

        You MUST double check your query before executing it. If you get an error while
        executing a query, rewrite the query and try again.

        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
        database.

        The tables in the database is given as {table_names}. To start you should ALWAYS look at the tables and their columns in the database to see what you
        can query. Do NOT skip this step.

        Then you should query the schema of the most relevant tables.
        """.format(
            dialect="SQLite",
            table_names=", ".join(table_names)
        )

        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        tools = toolkit.get_tools()
        agent_executor = create_react_agent(llm, tools, prompt=system_message)

        answer = agent_executor.invoke(
            {"messages": [{"role": "user", "content": item["question"]}]},
            config={"recursion_limit": 10}
        )

        generated_query = extract_sql_and_result_from_agent_output(answer)["generated_sql"]
        generated_result = extract_sql_and_result_from_agent_output(answer)["generated_result"]

        medium_answers.append({
            "db_id": item["db_id"],
            "question": item["question"],
            "generated_sql": generated_query or "",
            "result": generated_result or "",
            "gold_sql": item["query"],
            "gold_result": item["result"]
        })

        print(f"✅ Success at index {i}")
        print(medium_answers[i])

        with open(save_path, "w") as f:
            json.dump(medium_answers, f, indent=2)

    except Exception as e:
        error_msg = str(e)
        if "GRAPH_RECURSION_LIMIT" in error_msg or "Recursion limit" in error_msg:
            print(f"⚠️ Recursion limit hit at index {i}. Skipping.")
            medium_answers.append({
                "db_id": item["db_id"],
                "question": item["question"],
                "generated_sql": "",
                "result": "",
                "gold_sql": item["query"]
            })
            print(medium_answers)
        elif "tool_use_failed" in error_msg or "table" in error_msg.lower():
            print(f"⚠️ Tool failure or table issue at index {i}. Skipping.")
            break
        else:
            print(f"❌ Error at index {i}: {e}")
            break

        with open(save_path, "w") as f:
            json.dump(medium_answers, f, indent=2)

Generating SQL:   0%|          | 0/22 [00:00<?, ?q/s]

What are  the different countries with singers above age 20?


Generating SQL:   5%|▍         | 1/22 [00:06<02:25,  6.92s/q]

✅ Success at index 1
{'db_id': 'concert_singer', 'question': 'What are  the different countries with singers above age 20?', 'generated_sql': 'SELECT DISTINCT country FROM singer WHERE age > 20', 'result': "('France',), ('Netherlands',), ('United States',)", 'gold_sql': 'select distinct country from singer where age > 20', 'gold_result': "('Netherlands',), ('France',), ('United States',)"}
Return the owner id, first name and last name of the owner who has the most dogs.


Generating SQL:   9%|▉         | 2/22 [00:27<04:54, 14.72s/q]

...

✅ Success at index 20
{'db_id': 'course_teach', 'question': 'For each hometown, how many teachers are there?', 'generated_sql': 'SELECT hometown, COUNT(*) FROM teacher GROUP BY hometown', 'result': "('Blackrod Urban District', '1'), ('Bolton County Borough', '1'), ('Farnworth Municipal Borough', '1'), ('Horwich Urban District', '1'), ('Kearsley Urban District', '1'), ('Little Lever Urban District', '1'), ('Turton Urban District', '1')", 'gold_sql': 'select hometown, count(*) from teacher group by hometown', 'gold_result': "('Farnworth Municipal Borough', '1'), ('Horwich Urban District', '1'), ('Kearsley Urban District', '1'), ('Little Lever Urban District', '1'), ('Bolton County Borough', '1'), ('Blackrod Urban District', '1'), ('Turton Urban District', '1')"}
List the most common hometown of teachers.


Generating SQL:  95%|█████████▌| 21/22 [05:05<00:11, 11.83s/q]

✅ Success at index 21
{'db_id': 'course_teach', 'question': 'List the most common hometown of teachers.', 'generated_sql': 'SELECT hometown, COUNT(*) as count FROM teacher GROUP BY hometown ORDER BY count DESC LIMIT 1', 'result': "('Turton Urban District', '1')", 'gold_sql': 'select hometown from teacher group by hometown order by count(*) desc limit 1', 'gold_result': "('Turton Urban District',)"}
What is the most commmon hometowns for teachers?


Generating SQL: 100%|██████████| 22/22 [05:27<00:00, 14.87s/q]

✅ Success at index 22
{'db_id': 'course_teach', 'question': 'What is the most commmon hometowns for teachers?', 'generated_sql': 'SELECT hometown, COUNT(*) as count FROM teacher GROUP BY hometown ORDER BY count DESC LIMIT 10', 'result': "('Blackrod Urban District', '1'), ('Bolton County Borough', '1'), ('Farnworth Municipal Borough', '1'), ('Horwich Urban District', '1'), ('Kearsley Urban District', '1'), ('Little Lever Urban District', '1'), ('Turton Urban District', '1')", 'gold_sql': 'select hometown from teacher group by hometown order by count(*) desc limit 1', 'gold_result': "('Turton Urban District',)"}


## Easy Questions

In [60]:
import json
with open("extracted_sql_examples_easy.json", "r") as f:
    data = json.load(f)

golden_data_easy = [
    {"db_id": item["db_id"], "question": item["question"], "query": item["gold_sql"], "result": item["gold_result"]}
    for item in data
]

In [61]:
golden_data_easy[0]

{'db_id': 'concert_singer',
 'question': 'How many singers do we have?',
 'query': 'select count(*) from singer',
 'result': "('6',)"}

In [63]:
import json
import os
from tqdm import tqdm

save_path = "easy_answers.json"

# Load previous progress if available
if os.path.exists(save_path):
    with open(save_path, "r") as f:
        easy_answers = json.load(f)
else:
    easy_answers = []

start_index = len(easy_answers)

# Wrap tqdm around the remaining examples
for i in tqdm(range(start_index, len(golden_data_easy)), desc="Generating SQL", unit="q"):
    item = golden_data_easy[i]
    try:
        print(item["question"])
        system_message = """
        You are an agent designed to interact with a SQL database.
        Given an input question, create a syntactically correct {dialect} query to run,
        then look at the results of the query and return the answer.
        You can order the results by a relevant column to return the most interesting
        examples in the database. Never query for all the columns from a specific table,
        only ask for the relevant columns given the question.

        You MUST double check your query before executing it. If you get an error while
        executing a query, rewrite the query and try again.

        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
        database.

        The tables in the database is given as {table_names}. To start you should ALWAYS look at the tables and their columns in the database to see what you
        can query. Do NOT skip this step.

        Then you should query the schema of the most relevant tables.
        """.format(
        dialect="SQLite",
        table_names=db.get_usable_table_names()
        )
        db = SQLDatabase.from_uri(f"sqlite:////Users/onurcanmemis/Downloads/spider_data/database/{item['db_id']}/{item['db_id']}.sqlite")
        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        tools = toolkit.get_tools()
        agent_executor = create_react_agent(llm, tools, prompt=system_message)
        answer=agent_executor.invoke({"messages": [{"role": "user", "content": item["question"]}]},
                                     config={"recursion_limit": 10})
        #answers.update({"query":extract_sql_code_block(answer.get("messages")[-3].content),"return":extract_sql_code_block(answer.get("messages")[-2].content)})
        generated_query=extract_sql_and_result_from_agent_output(answer)["generated_sql"]
        generated_result=extract_sql_and_result_from_agent_output(answer)["generated_result"]
        easy_answers.append({
            "db_id": item["db_id"],
            "question": item["question"],
            "generated_sql": generated_query or "",
            "result": generated_result or "",
            "gold_sql":item["query"],
            "gold_result":item["result"]
        })
        print(f"✅ Success at index {i}")
        print(easy_answers[i])

        # Save progress after each successful query
        with open(save_path, "w") as f:
            json.dump(easy_answers, f, indent=2)

    except Exception as e:
        error_msg = str(e)
        if "GRAPH_RECURSION_LIMIT" in error_msg or "Recursion limit" in error_msg:
            print(f"⚠️ Recursion limit hit at index {i}. Skipping.")
            easy_answers.append({
            "db_id": item["db_id"],
            "question": item["question"],
            "generated_sql": "",
            "result": ""})
            print(easy_answers[i])
        elif "tool_use_failed" in error_msg or "table" in error_msg.lower():
            print(f"⚠️ Tool failure or table issue at index {i}. Skipping.")
            break
        else:
            print(f"❌ Error at index {i}: {e}")
            break

        with open(save_path, "w") as f:
            json.dump(easy_answers, f, indent=2)

Generating SQL:   0%|          | 0/48 [00:00<?, ?q/s]

How many singers do we have?


Generating SQL:   2%|▏         | 1/48 [00:12<10:08, 12.96s/q]

✅ Success at index 0
{'db_id': 'concert_singer', 'question': 'How many singers do we have?', 'generated_sql': "SELECT COUNT(*) FROM teacher WHERE role = 'singer'", 'result': "Error: (sqlite3.OperationalError) no such table: teacher\n[SQL: SELECT COUNT(*) FROM teacher WHERE role = 'singer']\n(Background on this error at: https://sqlalche.me/e/20/e3q8)", 'gold_sql': 'select count(*) from singer', 'gold_result': "('6',)"}
What is the total number of singers?


Generating SQL:   4%|▍         | 2/48 [00:31<12:24, 16.18s/q]

...

✅ Success at index 45
{'db_id': 'employee_hire_evaluation', 'question': 'Find the total amount of bonus given in all the evaluations.', 'generated_sql': 'SELECT SUM(bonus) FROM evaluation', 'result': "('19500.0',)", 'gold_sql': 'select sum(bonus) from evaluation', 'gold_result': "('19500.0',)"}
Give me all the information about hiring.


Generating SQL:  98%|█████████▊| 47/48 [11:44<00:13, 13.96s/q]

✅ Success at index 46
{'db_id': 'employee_hire_evaluation', 'question': 'Give me all the information about hiring.', 'generated_sql': '', 'result': '', 'gold_sql': 'select * from hiring', 'gold_result': "('4', '4', '2012', 'T'), ('1', '1', '2009', 'T'), ('2', '6', '2010', 'F'), ('8', '3', '2011', 'F'), ('6', '7', '2008', 'T'), ('1', '2', '2003', 'T'), ('5', '5', '2013', 'T')"}
What is all the information about hiring?


Generating SQL: 100%|██████████| 48/48 [12:08<00:00, 15.18s/q]

✅ Success at index 47
{'db_id': 'employee_hire_evaluation', 'question': 'What is all the information about hiring?', 'generated_sql': '', 'result': '', 'gold_sql': 'select * from hiring', 'gold_result': "('4', '4', '2012', 'T'), ('1', '1', '2009', 'T'), ('2', '6', '2010', 'F'), ('8', '3', '2011', 'F'), ('6', '7', '2008', 'T'), ('1', '2', '2003', 'T'), ('5', '5', '2013', 'T')"}


In [64]:
import json
import ast

# Load the merged JSON file
with open("easy_answers.json", "r") as f:
    data = json.load(f)

# Utility to parse result string into list of tuples
def parse_result_string(result_str):
    try:
        parsed = ast.literal_eval(f"[{result_str}]")  # convert comma-separated tuples into a list
        return parsed if isinstance(parsed, list) else []
    except Exception:
        return []

# Build prediction and gold lists
predictions = []
gold_data = []

for item in data:
    pred_sql = item.get("generated_sql", "")
    pred_result = parse_result_string(item.get("result", ""))

    gold_sql = item.get("gold_sql", "")
    gold_result = parse_result_string(item.get("gold_result", ""))

    predictions.append((pred_sql, pred_result))
    gold_data.append((gold_sql, gold_result))

# Check samples
print("Example prediction:", predictions[0])
print("Example gold:", gold_data[0])

Example prediction: ("SELECT COUNT(*) FROM teacher WHERE role = 'singer'", [])
Example gold: ('select count(*) from singer', [('6',)])


## Evaluation

In [57]:
from typing import List, Tuple, Dict
import sqlparse

def normalize_sql(sql):
    sql = sql.strip().rstrip(';')
    parsed = sqlparse.format(sql, keyword_case='lower', strip_comments=True, reindent=True)
    return " ".join(parsed.strip().split()).lower()

def normalize_result(result):
    if not result or not isinstance(result, list):
        return set()

    normalized = set()
    for row in result:
        if row is None:
            continue  # skip invalid rows
        try:
            normalized.add(tuple(str(item).strip() for item in row))
        except Exception:
            continue  # skip malformed rows

    return normalized
def extract_components(sql):
    parsed = sqlparse.parse(sql)
    if not parsed:
        return {"select": set(), "from": set(), "where": set()}
    stmt = parsed[0]

    select_tokens = set()
    from_tokens = set()
    where_tokens = set()

    is_select = False
    is_from = False
    is_where = False

    for token in stmt.tokens:
        if token.is_group:
            for subtoken in token.flatten():
                tval = subtoken.value.lower().strip()
                if tval in ("select", "from", "where"):
                    is_select = tval == "select"
                    is_from = tval == "from"
                    is_where = tval == "where"
                    continue
                if is_select and subtoken.ttype in (sqlparse.tokens.Name, sqlparse.tokens.Wildcard):
                    select_tokens.add(tval)
                elif is_from and subtoken.ttype in (sqlparse.tokens.Name,):
                    from_tokens.add(tval)
                elif is_where and subtoken.ttype in (sqlparse.tokens.Name, sqlparse.tokens.Literal.Number.Integer, sqlparse.tokens.Operator.Comparison):
                    where_tokens.add(tval)
        else:
            tval = token.value.lower().strip()
            if tval in ("select", "from", "where"):
                is_select = tval == "select"
                is_from = tval == "from"
                is_where = tval == "where"
                continue
            if is_select and token.ttype in (sqlparse.tokens.Name, sqlparse.tokens.Wildcard):
                select_tokens.add(tval)
            elif is_from and token.ttype in (sqlparse.tokens.Name,):
                from_tokens.add(tval)
            elif is_where and token.ttype in (sqlparse.tokens.Name, sqlparse.tokens.Literal.Number.Integer, sqlparse.tokens.Operator.Comparison):
                where_tokens.add(tval)

    return {"select": select_tokens, "from": from_tokens, "where": where_tokens}

def jaccard_similarity(set1, set2):
    if not set1 and not set2:
        return 1.0
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0
def evaluate_predictions(
    predictions: List[Tuple[str, List[Tuple]]],
    gold_data: List[Tuple[str, List[Tuple]]]
) -> Dict[str, float]:
    """
    Evaluate a list of predicted SQL queries and their results against gold SQL queries and results.

    Each element in the predictions and gold_data lists is a tuple:
    (sql_query: str, result: List[Tuple])

    Returns:
    A dictionary with evaluation metrics.
    """
    exact_match_count = 0
    execution_match_count = 0
    total = len(predictions)

    jaccard_scores = {
        "select": 0.0,
        "from": 0.0,
        "where": 0.0
    }

    for (pred_sql, pred_result), (gold_sql, gold_result) in zip(predictions, gold_data):
        norm_pred_sql = normalize_sql(pred_sql)
        norm_gold_sql = normalize_sql(gold_sql)

        norm_pred_result = normalize_result(pred_result)
        norm_gold_result = normalize_result(gold_result)

        if norm_pred_sql == norm_gold_sql:
            exact_match_count += 1

        if norm_pred_result == norm_gold_result:
            execution_match_count += 1

        pred_components = extract_components(norm_pred_sql)
        gold_components = extract_components(norm_gold_sql)

        for key in jaccard_scores.keys():
            jaccard_scores[key] += jaccard_similarity(pred_components.get(key, set()), gold_components.get(key, set()))

    # Average jaccard scores
    for key in jaccard_scores:
        jaccard_scores[key] /= total

    metrics = {
        "total": total,
        "exact_match_accuracy": exact_match_count / total if total else 0.0,
        "execution_match_accuracy": execution_match_count / total if total else 0.0,
        "select_jaccard": jaccard_scores["select"],
        "from_jaccard": jaccard_scores["from"],
        "where_jaccard": jaccard_scores["where"]
    }

    return metrics

In [127]:
import json
import ast

# Load the merged JSON file
with open("hard_answers.json", "r") as f:
    data = json.load(f)

# Utility to parse result string into list of tuples
def parse_result_string(result_str):
    try:
        parsed = ast.literal_eval(f"[{result_str}]")  # convert comma-separated tuples into a list
        return parsed if isinstance(parsed, list) else []
    except Exception:
        return []

# Build prediction and gold lists
hard_predictions = []
hard_gold_data = []

for item in data:
    pred_sql = item.get("generated_sql", "")
    pred_result = parse_result_string(item.get("result", ""))

    gold_sql = item.get("gold_sql", "")
    gold_result = parse_result_string(item.get("gold_result", ""))

    hard_predictions.append((pred_sql, pred_result))
    hard_gold_data.append((gold_sql, gold_result))

# Check samples
print("Example prediction:", hard_predictions[0])
print("Example gold:", hard_gold_data[0])

Example prediction: ('SELECT T1.country FROM singer AS T1 INNER JOIN singer_in_concert AS T2 ON T1.id = T2.singer_id INNER JOIN concert AS T3 ON T2.concert_id = T3.id WHERE T1.age > 40 INTERSECT SELECT T1.country FROM singer AS T1 INNER JOIN singer_in_concert AS T2 ON T1.id = T2.singer_id INNER JOIN concert AS T3 ON T2.concert_id = T3.id WHERE T1.age < 30', [])
Example gold: ('select country from singer where age > 40 intersect select country from singer where age < 30', [('France',)])


In [128]:
evaluate_predictions(hard_predictions, hard_gold_data)

{'total': 15,
 'exact_match_accuracy': 0.13333333333333333,
 'execution_match_accuracy': 0.6666666666666666,
 'select_jaccard': 0.8111111111111111,
 'from_jaccard': 0.674074074074074,
 'where_jaccard': 0.88}

In [129]:
import json
import ast

# Load the merged JSON file
with open("medium_answers.json", "r") as f:
    data = json.load(f)

# Utility to parse result string into list of tuples
def parse_result_string(result_str):
    try:
        parsed = ast.literal_eval(f"[{result_str}]")  # convert comma-separated tuples into a list
        return parsed if isinstance(parsed, list) else []
    except Exception:
        return []

# Build prediction and gold lists
medium_predictions = []
medium_gold_data = []

for item in data:
    pred_sql = item.get("generated_sql", "")
    pred_result = parse_result_string(item.get("result", ""))

    gold_sql = item.get("gold_sql", "")
    gold_result = parse_result_string(item.get("gold_result", ""))

    medium_predictions.append((pred_sql, pred_result))
    medium_gold_data.append((gold_sql, gold_result))

# Check samples
print("Example prediction:", medium_predictions[0])
print("Example gold:", medium_gold_data[0])

Example prediction: ('SELECT DISTINCT country FROM singer WHERE age > 20', [('France',), ('Netherlands',), ('United States',)])
Example gold: ('select distinct country from singer where age > 20', [('Netherlands',), ('France',), ('United States',)])


In [130]:
evaluate_predictions(medium_predictions, medium_gold_data)

{'total': 23,
 'exact_match_accuracy': 0.7391304347826086,
 'execution_match_accuracy': 0.782608695652174,
 'select_jaccard': 0.9043478260869564,
 'from_jaccard': 0.9195134575569359,
 'where_jaccard': 1.0}

In [131]:
import json
import ast

# Load the merged JSON file
with open("easy_answers.json", "r") as f:
    data = json.load(f)

# Utility to parse result string into list of tuples
def parse_result_string(result_str):
    try:
        parsed = ast.literal_eval(f"[{result_str}]")  # convert comma-separated tuples into a list
        return parsed if isinstance(parsed, list) else []
    except Exception:
        return []

# Build prediction and gold lists
easy_predictions = []
easy_gold_data = []

for item in data:
    pred_sql = item.get("generated_sql", "")
    pred_result = parse_result_string(item.get("result", ""))

    gold_sql = item.get("gold_sql", "")
    gold_result = parse_result_string(item.get("gold_result", ""))

    easy_predictions.append((pred_sql, pred_result))
    easy_gold_data.append((gold_sql, gold_result))

# Check samples
print("Example prediction:", easy_predictions[0])
print("Example gold:", easy_gold_data[0])

Example prediction: ("SELECT COUNT(*) FROM teacher WHERE role = 'singer'", [])
Example gold: ('select count(*) from singer', [('6',)])


In [132]:
easy_predictions

[("SELECT COUNT(*) FROM teacher WHERE role = 'singer'", []),
 ('SELECT COUNT(*) FROM singer', [('6',)]),
 ('SELECT name, country, age FROM singer ORDER BY age DESC',
  [('Joe Sharp', 'Netherlands', '52'),
   ('John Nizinik', 'France', '43'),
   ('Justin Brown', 'France', '29'),
   ('Rose White', 'France', '41'),
   ('Timbaland', 'United States', '32'),
   ('Tribal King', 'France', '25')]),
 ('SELECT COUNT(*) FROM singer', [('8',)]),
 ('SELECT COUNT(*) FROM singer', [('8',)]),
 ('SELECT birth_year, citizenship FROM singer',
  [('1940.0', 'United States'),
   ('1942.0', 'Chile'),
   ('1944.0', 'France'),
   ('1948.0', 'United States'),
   ('1949.0', 'United States'),
   ('1953.0', 'Australia'),
   ('1961.0', 'United States'),
   ('1962.0', 'Germany')]),
 ('SELECT birth_year, citizenship FROM singer',
  [('1940.0', 'United States'),
   ('1942.0', 'Chile'),
   ('1944.0', 'France'),
   ('1948.0', 'United States'),
   ('1949.0', 'United States'),
   ('1953.0', 'Australia'),
   ('1961.0', 'Un

In [133]:
evaluate_predictions(easy_predictions, easy_gold_data)

{'total': 48,
 'exact_match_accuracy': 0.5416666666666666,
 'execution_match_accuracy': 0.6458333333333334,
 'select_jaccard': 0.859375,
 'from_jaccard': 0.7673611111111112,
 'where_jaccard': 0.9583333333333334}

## Logic Equivalence Check

In [134]:
import json

# Helper function
def extract_question_and_result(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()

    data = []
    current = {}

    for line in lines:
        line = line.strip()
        if line.startswith("Question:"):
            current["question"] = line.replace("Question:", "").strip()
        elif line.startswith("Execution Result:"):
            current["execution_result"] = line.replace("Execution Result:", "").strip()
            if "question" in current:  # Ensure both parts exist
                data.append(current)
            current = {}  # Reset for next instance

    return data

# Process each difficulty file
easy_data = extract_question_and_result("easy.txt")
medium_data = extract_question_and_result("medium.txt")
hard_data = extract_question_and_result("hard.txt")

# Write to JSON
with open("questions_easy.json", "w") as f:
    json.dump(easy_data, f, indent=2)

with open("questions_medium.json", "w") as f:
    json.dump(medium_data, f, indent=2)

with open("questions_hard.json", "w") as f:
    json.dump(hard_data, f, indent=2)

print("✅ JSON files written with question and execution_result.")

✅ JSON files written with question and execution_result.


In [135]:
with open("questions_easy.json", "r") as f:
    easy_data = json.load(f)

with open("questions_medium.json", "r") as f:
    medium_data = json.load(f)

with open("questions_hard.json", "r") as f:
    hard_data = json.load(f)

print("Number of easy questions:", len(easy_data))
print("Number of medium questions:", len(medium_data))
print("Number of hard questions:", len(hard_data))

Number of easy questions: 48
Number of medium questions: 23
Number of hard questions: 15


In [140]:
easy_execution=[]
medium_execution=[]
hard_execution=[]

for item in easy_predictions:
    easy_execution.append(item[1])

for item in medium_predictions:
    medium_execution.append(item[1])

for item in hard_predictions:
    hard_execution.append(item[1])

easy_execution=[str(item) for item in easy_execution]
medium_execution=[str(item) for item in medium_execution]
hard_execution=[str(item) for item in hard_execution]

In [137]:
custom_easy=[]
custom_medium=[]
custom_hard=[]

for item in easy_data:
    custom_easy.append(item["execution_result"].replace("{", "[").replace("}", "]"))

for item in medium_data:
    custom_medium.append(item["execution_result"].replace("{", "[").replace("}", "]"))

for item in hard_data:
    custom_hard.append(item["execution_result"].replace("{", "[").replace("}", "]"))

In [148]:
import ast

def normalize_result_string(result_str):
    """
    Converts a result string like "[('Alice',), ('Bob',)]" or "('Alice',), ('Bob',)" into a set of tuples.
    Handles empty strings and malformed inputs gracefully.
    """
    if not result_str.strip():
        return set()

    try:
        # Ensure it's parsable as a list: wrap if not
        if not result_str.strip().startswith("["):
            result_str = "[" + result_str + "]"

        parsed = ast.literal_eval(result_str)
        if isinstance(parsed, list):
            return set(tuple(str(x).strip() for x in row) for row in parsed)
    except Exception:
        pass

    return set()

def logic_equivalence_score(langchain_results, agentic_results):
    assert len(langchain_results) == len(agentic_results), "Mismatched result lengths"

    total = len(langchain_results)
    equivalent_count = 0

    for l_result, a_result in zip(langchain_results, agentic_results):
        if normalize_result_string(l_result) == normalize_result_string(a_result):
            equivalent_count += 1

    return equivalent_count / total if total > 0 else 0.0

In [149]:
logic_equivalence_score(easy_execution,custom_easy)

0.6458333333333334

In [150]:
logic_equivalence_score(medium_execution,custom_medium)

0.782608695652174

In [151]:
logic_equivalence_score(hard_execution,custom_hard)

0.6